# Instagram Graph API call

## 1. Get Credential (w/ dotenv)

To protect credential info, you have to make `.env` file  
Here's some examples  

1. INSTAGRAM_ACCESS_TOKEN
2. INSTAGRAM_BUSINESS_ACCOUNT
3. FB_APP_SECRET_CODE
4. FB_CLIENT_ID

In [37]:
import requests
import json
import os
from pprint import pprint as pp
from dotenv import load_dotenv

# load .env
load_dotenv()
IG_ACCESS_TOKEN=os.environ.get("IG_ACCESS_TOKEN")
# FB_APP_SECRET_CODE=os.environ.get('FB_APP_SECRET_CODE')
IG_BUSINESS_ID=os.environ.get('IG_BUSINESS_ID')
IG_USER_NAME=os.environ.get('IG_USER_NAME')

def get_creds():
    """
    {'access_token': '',
    'graph_domain': 'https://graph.facebook.com/',
    'graph_version': 'v20.0',
    'instagram_account_id': '',
    'ig_username': '',
    'endpoint_base': 'https://graph.facebook.com/v20.0/'}

    """
    creds = dict() 
    creds['access_token'] = IG_ACCESS_TOKEN   # access token for use with all api calls
    # creds['client_secret'] = FB_APP_SECRET_CODE  # client secret from facebook app
    creds['graph_domain'] = 'https://graph.facebook.com/' # base domain for api calls
    creds['graph_version'] = 'v20.0' # version of api we are hitting
    creds['instagram_account_id'] = IG_BUSINESS_ID # user business account id
    creds['ig_username'] = IG_USER_NAME
    # with domain and version
    creds['endpoint_base'] = creds['graph_domain'] + creds['graph_version'] + '/' # base endpoint
    
    return creds

In [170]:
params = get_creds()
params

{'access_token': 'EAAMZCN23AZACwBOzTLIZCIlNokpsJ8aN5fH9torpU9hQMj44bL9I4uZBHPxlDFX5mQ1uL9E9DwGd1yfEOl2hbKSeGP2nt2QGD2k2jkGTdLtc5JWfNtrar1FSmtBRwx1SMIBdbxH0qoW2fHJrJTzWHpWVax3dSYAUF1Fhi2ZCfCG0GberWhXCs5kV7',
 'graph_domain': 'https://graph.facebook.com/',
 'graph_version': 'v20.0',
 'instagram_account_id': '17841467510518283',
 'ig_username': 'analyst_hyuk',
 'endpoint_base': 'https://graph.facebook.com/v20.0/'}

## 2. Create Media Object Response & Publish Media

### (Optional) Create Carousel post

Steps required for making a carousel post are.  
1. Upload every images or video you want to make a carousel of one by one and store the container_id in a list.  
2. Make another container that contains the container_id list.  
3. Publish the container.  

In [ ]:
import requests

def create_media(params, include_caption=True, include_children=False):
    """Create media container object

    API Endpoint:
    - For image
        https://graph.facebook.com/v20.0/{ig-user-id}/media?image_url={image-url}&caption={caption}&access_token={access_token}
    - For video
        https://graph.facebook.com/v20.0/{ig-user-id}/media?media_type=VIDEO&video_url={video-url}&caption={caption}&access_token={access_token}
    - For Carousel
        https://graph.facebook.com/v20.0/{ig-user-id}/media?media_type=CAROUSEL&children={children}&caption={caption}&access_token={access_token}

    Returns:
        json object: {id : media_container_object_id}
    """

    url = params['endpoint_base'] + params['instagram_account_id'] + '/media'
    
    endpointParams = {
        'access_token': params['access_token']
    }

    if params['media_type'] == 'IMAGE':
        endpointParams['image_url'] = params['media_url']
    
    elif params['media_type'] == 'VIDEO':
        endpointParams['video_url'] = params['media_url']
        endpointParams['media_type'] = params['media_type']
    
    # media_type == 'CAROUSEL'
    else:
        endpointParams['media_type'] = params['media_type']

    # for single media post
    if include_caption:
        endpointParams['caption'] = params['caption']
        
    # for carousel.
    if include_children:
        endpointParams['children'] = params['children']
        response = requests.post(url, json=endpointParams)
        return response.json()

    response = requests.post(url, data=endpointParams)
    return response.json()

def create_single_media(params):
    """Create `single media` container object"""
    return create_media(params, include_caption=True)

def create_multi_media(params):
    """Create `multi media` container object"""
    return create_media(params, include_caption=False)

def create_slide_object(params):
    """Create `slide media` container object"""
    return create_media(params, include_caption=True, include_children=True)

def publish_media(media_object_id, params):
    """ Publish `media container` object
    
    API Endpoint:
        https://graph.facebook.com/
        v20.0/{ig-user-id}/media_publish?creation_id={creation-id}&access_token={access_token}

    Returns:
        json object: {id: media_id}
    """

    url = params['endpoint_base'] + params['instagram_account_id'] + '/media_publish'

    endpointParams = dict()
    endpointParams['creation_id'] = media_object_id
    endpointParams['access_token'] = params['access_token']

    response = requests.post(url, data=endpointParams)
    
    print("\n---- MEDIA OBJECT PUBLISHED ---- \n")
    print("\tID:") # label
    print("\t" + response.json()['id']) # id of the object

    return response.json()


In [ ]:
def post_media(queue:pd.DataFrame):
    """ Run post process

    Sequence
        1. read_csv ('queue.csv')
        2. post to instagram - with using 1st record
        3. return csv
            - len(queue) >= 1 (pop first record and return remains)
            - len(queue) == 0 (return blank queue)
    """

    # queue의 데이터가 있으면 진행
    if len(queue):
        data = queue.iloc[0]

        params = get_creds()
        params['caption'] = data['caption']

        # single_media upload
        if (len(data['media_url']) == 1):
            params['media_url'], params['media_type'] = data['media_url'][0]
            media_container = create_single_media(params)

        # multi_media upload
        else:
            print('here')
            media_id_list = []

            # media container 생성 - 최대 10개까지만 업로드 가능
            for (media_url, media_type) in data['media_url'][:10]:
                params['media_type'] = media_type
                params['media_url'] = media_url
                temp_media_container = create_multi_media(params)
                media_id_list.append(temp_media_container['id'])
                
            # carousel로 넘기기
            params['media_type'] = 'CAROUSEL'
            params['children'] = media_id_list
            media_container = create_slide_object(params)
        
        # published 된 경우 object published 출력
        publish_media(media_container['id'], params) 
        
        # post comment (Question)
        # 글 생성과 동시에 Question 생성하는 경우
        pass

        # post reply to comment (Answer)
        # 글 생성과 동시에 Q&A 까지 함께 진행하는 경우
        pass

        # 작업 종료 후, 맨 위에서 하나 빼고 csv저장
        queue = queue.drop(index=0).reset_index(drop=True)  # 0번 인덱스 드랍
        queue.to_csv('queue.csv', encoding='utf-8', index=False) # 값 저장.
        print(f'\tRemain records: {len(queue)}')
        return 
    
    # queue에 데이터가 없으면 바로 종료
    else:
        print("\n---- Sorry, Check file plz :D ---- \n")
        print(f'\tNothing to post in `queue.csv`')
        return


### 샘플 인풋 데이터 생성 (`queue.csv`)
- queue['media_url'] 형식 (url, media_type) 으로 변경

In [1]:
import pandas as pd

# Create sample data
df = pd.DataFrame(columns=['upload_time', 'media_url', 'caption', 'Q_comment', 'A_reply'])
urls = [('https://octapi.lxzin.com/interior/vImgFileSeq/202210/11/8ede80a1-1d0c-4839-bcc3-97bd4f357ecd.jpg', 'IMAGE'),('https://octapi.lxzin.com/interior/vImgFileSeq/202210/11/8ede80a1-1d0c-4839-bcc3-97bd4f357ecd.jpg', 'IMAGE')]
temp_str = '\n'*20

# carousel (multi-data)
data_multi = pd.DataFrame({
    'upload_time': [None],
    'media_url': [urls],
    'caption' : ['multi_media_test'],
    'Q_comment': ['문제를 맞춰봅시다 ^&^'],
    'A_reply' : [f'정답을 알아봐요! {temp_str} O~X~']})

# single media
single_url = [('https://octapi.lxzin.com/interior/vImgFileSeq/202210/11/8ede80a1-1d0c-4839-bcc3-97bd4f357ecd.jpg', 'IMAGE')]
data_single = pd.DataFrame({
    'upload_time': [None],
    'media_url': [single_url], # (url, media_type)
    'caption' : ['single_media_test'],
    'Q_comment': ['no_comments'],
    'A_reply' : [None]})

queue_df = pd.concat([data_multi, data_single], axis=0, ignore_index=True)
queue_df.to_csv('queue.csv', encoding='utf-8', index=False)


In [2]:
# 데이터 불러오기
import ast
queue = pd.read_csv('queue.csv', encoding='utf-8', converters={'media_url' : ast.literal_eval}) 
queue

,upload_time,media_url,caption,Q_comment,A_reply
0,NaN,[(https://octapi.lxzin.com/interior/vImgFileSe...,multi_media_test,문제를 맞춰봅시다 ^&^,정답을 알아봐요! \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1,NaN,[(https://octapi.lxzin.com/interior/vImgFileSe...,single_media_test,no_comments,NaN


In [237]:
# queue.csv를 읽은 후, df상태로 넣어주면 자동 처리.
post_media(queue)


---- Sorry, Check file plz :D ---- 

	Nothing to post in `queue.csv`


#### Example Post

In [73]:
# # Create Image Media Post
# params=get_creds()
# params['media_type'] = 'IMAGE' # 2types in input('image', 'video')
# params['media_url'] = 'https://t2.daumcdn.net/thumb/R720x0/?fname=http://t1.daumcdn.net/brunch/service/user/2fG8/image/GUbIfYW-1qfxrzemAv6EHv8XP6M.jpg'

# image_media_object_1 = create_multi_media(params)
# image_media_object_1

{'id': '17893437330046479'}

In [69]:
# # Create Image Media Post
# params=get_creds()
# params['media_type'] = 'IMAGE' # 2types in input('image', 'video')
# params['media_url'] = 'https://octapi.lxzin.com/interior/vImgFileSeq/202210/11/8ede80a1-1d0c-4839-bcc3-97bd4f357ecd.jpg'

# image_media_object_2 = create_multi_media(params)

In [39]:
# def create_slide_object(params):
#     """Create media `Container` object
    
#     API Endpoint:
#     - For image
#         https://graph.facebook.com/
#         v20.0/{ig-user-id}/media?image_url={image-url}&caption={caption}&access_token={access_token}
    
#     - For video
#         https://graph.facebook.com/
#         v20.0/{ig-user-id}/media?video_url={video-url}&caption={caption}&access_token={access_token}

#     Returns:
#         json object: {id : media_container_object_id}
#     """

#     url = params['endpoint_base'] + params['instagram_account_id'] + '/media'

#     endpointParams = dict()
#     endpointParams['media_type'] = params['media_type']
#     endpointParams['caption'] = params['caption']
#     endpointParams['children'] = params['children']
#     endpointParams['access_token'] = params['access_token']
    
#     response = requests.post(url, json=endpointParams)

#     return response.json()

In [75]:
# # Create Image Media Post
# params=get_creds()
# params['media_type'] = 'CAROUSEL' # 2types in input('image', 'video')
# params['caption'] = 'MY CATS'
# params['children'] = [image_media_object_1['id'], image_media_object_2['id']]

# image_media_object_carousel = create_slide_object(params)
# image_media_object_carousel

{'id': '18032762555297091'}

## 3. Get User Media & Comments

In [14]:
def get_user_media(params):
    """ Get User Media id
    
    API Endpoint:
        https://graph.facebook.com/
        v20.0/{ig-user-id}?fields=media&access_token={access_token}

    Returns:
        json object:
    
        {'data' : [{id : {media_id_1},
                    id : {media_id_2},
                    id : {media_id_3},]}
    """
    
    url = params['endpoint_base'] + params['instagram_account_id'] + '/media'
    
    endpointParams = dict()
    endpointParams['access_token'] = params['access_token']
    endpointParams['fields'] = 'media'
    response = requests.get(url, endpointParams)
    return response.json()

def get_media_comments(media_id, params):
    """  Get media comments

    API Endpoint:
        https://graph.facebook.com/
        v20.0/{media_id}/comments?access_token={access_token}
    
    Returns:
        json object:

        {'data' : [{'timestamp': {created_time},
                    'text': {comment_text_1},
                    'id' : {comment_id_1}},

                    {'timestamp': {created_time},
                    'text': {comment_text_2},
                    'id': {comment_id_2}}]}
    """
    url = params['endpoint_base'] + media_id + '/comments?'
    endpointParams = dict()
    endpointParams['access_token'] = params['access_token']

    response = requests.get(url, endpointParams)
    return response.json()

In [15]:
from pprint import pprint as pp

my_media = get_user_media(params)

# get all media comments
comments_data = []

for data in my_media['data']:
    temp_media_id = data['id']
    comments_data.append(get_media_comments(temp_media_id, params))

pp(comments_data)

[{'data': []},
 {'data': [{'id': '17915879570957592',
            'text': '안녕하세요',
            'timestamp': '2024-07-18T10:31:38+0000'}]},
 {'data': []},
 {'data': []},
 {'data': []},
 {'data': []},
 {'data': []},
 {'data': []},
 {'data': [{'id': '17874199341067145',
            'text': '멋져요! 누구인가요?',
            'timestamp': '2024-07-18T04:54:02+0000'}]},
 {'data': [{'id': '17912313272967135',
            'text': '귀여운 고영희 !',
            'timestamp': '2024-07-18T04:21:19+0000'}]},
 {'data': []}]


In [17]:
comments_id = []

for media_comments in comments_data:
    for comment in media_comments['data']:
        comments_id.append(comment['id'])
    
comments_id

['17915879570957592', '17874199341067145', '17912313272967135']

## 4. Post Reply for `comment`

### Reply Sequence
- 1. Get replies in comment
- 2. Check - Is it already replied?
  - If yes) Is the last response was created by media creator? 
- 3. Get all texts in replies (Optional - TBD)
- 4. Create reply response (w/ LLM)
- 5. Post reply to a comment

In [46]:
def get_comment_replies(comment_id, params):
    """Create reply for comment
    
    API Endpoint:
        https://graph.facebook.com/
        v20.0/{ig_comment_id}/replies?fields={fields}&access_token={access-token}
    
    Returns:
        json object: the replies are `stacked-shaped`(LIFO)
        
        {'data': [{'id': {replied_id_2},
                'text': {replied_text_2}
                'username': {ig_username},

                {'id', {replied_id_1},
                'text': {replied_text_1},
                'username': {ig_username}
                ]}
    """

    url = params['endpoint_base'] + comment_id + '/replies'
    endpointParams = dict()
    endpointParams['access_token'] = params['access_token']
    endpointParams['fields'] = 'id, text, username'
    response = requests.get(url, endpointParams)
    return response.json()

In [51]:
def is_replied(replies, params):
    """Check if last response's username is creator's name

    Returns:
        True or False
    """
    if len(replies['data']):
        return True if replies['data'][0]['username'] == params['ig_username'] else False

In [58]:
print(temp_replies)
print(is_replied(temp_replies, params))

{'data': [{'id': '17997719987643254', 'text': '태그 없이도 답글이 가능할까?', 'username': 'analyst_hyuk'}, {'id': '17970962732744635', 'text': 'This reply was created through API call', 'username': 'analyst_hyuk'}]}
True


In [60]:
def get_reply_texts(replies, params):
    """
        일단 그냥 LLM에 넣어보고 잘 돌아가면 pass,
        - 필요할 경우 이후 작성
    """
    pass

In [33]:
def post_reply_to_comment(comment_id, params):
    """ Post reply to a comment

    API Endpoint:
        https://graph.facebook.com/
        v20.0/{ig_comment_id}/replies&access_token={access-token}

    Returns:
        Status Code <Response>
        - 200 : OK
        - 400 : Not supported input form
    """
    
    url = params['endpoint_base'] + comment_id + '/replies'

    endpointParams = dict()
    endpointParams['message'] = params['reply_message']
    endpointParams['access_token'] = params['access_token']
    response = requests.post(url, data=endpointParams)
    return response

In [77]:
params['reply_message'] = "This reply was created through API call"

# post_response = post_reply_to_comment(comments_id[0], params)

In [61]:
# 참고문서
"""
https://developers.facebook.com/docs/instagram-platform/instagram-graph-api/content-publishing#----------            # publishing

https://developers.facebook.com/docs/instagram-platform/instagram-graph-api/reference/ig-user/media_publish         # replied comment

https://velog.io/@shi9476/Instagram-API-%EC%97%B0%EB%8F%99               # 참고자료 1 - long live token 자동재발급

https://medium.com/@maanideeprkummiitha/a-beginners-guide-to-interacting-with-instagram-using-python-and-the-graph-api-8fe7956fcc   # create api function tutorial

https://developers.facebook.com/docs/instagram-platform/instagram-graph-api/comment-moderation""" # 댓글에 답글 작성


'\nhttps://medium.com/@maanideeprkummiitha/a-beginners-guide-to-interacting-with-instagram-using-python-and-the-graph-api-8fe7956fcc   # create api function tutorial\n\nhttps://velog.io/@shi9476/Instagram-API-%EC%97%B0%EB%8F%99               # 참고자료 1 - long live token 자동재발급\n\nhttps://developers.facebook.com/docs/instagram-platform/instagram-graph-api/content-publishing#----------            # publishing\n\nhttps://developers.facebook.com/docs/instagram-platform/instagram-graph-api/reference/ig-user/media_publish         # replied comment\n\nhttps://developers.facebook.com/docs/instagram-platform/instagram-graph-api/comment-moderation'

In [22]:
def post_comment(media_id, params):
    """Post comment to media
    
    API Endpoint:
        https://graph.facebook.com/
        v20.0/{ig_comment_id}/comments?message={message}&access_token={access_token}


    """
    url= params['endpoint_base'] + media_id + '/comments'
    endpointParams= dict()
    endpointParams['message']= params['message']
    endpointParams['access_token']= params['access_token']
    
    response= requests.post(url, data= endpointParams)
    
    return response.json()

In [23]:
params['message']= '맞팔해요 ^&^'

post_response= post_comment(temp_media_id, params)